In [3]:
from tope import Tope
from tope.net import *
from tope.orth import *
from tope.graph import Graph
from tests import normalize_polygon

import json, os, numpy as np
with open("polys.json") as fd: polys = json.load(fd)

In [4]:
import stl

In [2]:
v_24cell = [
    [1, 0, 0, 0], [-1, 0, 0, 0],
    [0, 1, 0, 0], [0, -1, 0, 0],
    [0, 0, 1, 0], [0, 0, -1, 0], 
    [0, 0, 0, 1], [0, 0, 0, -1],
    [1, -1, 0, 0], [-1, 1, 0, 0], 
    [1, 0, -1, 0], [-1, 0, 1, 0], 
    [1, 0, 0, -1], [-1, 0, 0, 1], 
    [0, 1, -1, 0], [0, -1, 1, 0], 
    [0, 1, 0, -1], [0, -1, 0, 1],
    [1, 0, -1, -1], [-1, 0, 1, 1], 
    [0, 1, -1, -1], [0, -1, 1, 1],
    [1, 1, -1, -1], [-1, -1, 1, 1]
]

In [1]:
visible_faces = {59, 56, 42, 55, 56, 29, 52, 20, 63, 85, 62, 92, 68, 98, 50, 53, 8, 20, 15, 52, 20, 53, 62, 59, 49, 54, 24, 57, 29, 26, 84, 85, 92, 91, 83, 86, 67, 68, 73, 74, 61, 90, 89, 70, 5, 46, 69, 93, 94, 72, 73, 48, 32, 31,23,17, 19, 57, 17, 41, 46, 22, 19, 32, 30, 70, 36, 56, 59, 55, 63, 62, 50, 8, 13, 7, 52, 12, 15, 20, 7, 10, 18, 16, 18, 36, 27, 26, 29, 92, 86, 79, 89, 77, 78, 65, 37, 38, 43, 10, 11, 75, 34, 82, 49, 5, 13}

In [30]:
P = Tope.from_vertices(v_24cell)
node_labels = dict(enumerate(P.faces[-1]))


def gemma_pair(i, j):
    s = set.intersection(P.faces[-1][i], P.faces[-1][j])
    
    return s if s in P.faces[-2] and not (P.faces[-2].index(s) in visible_faces) else None

G = Graph.from_pairing(node_labels, gemma_pair, node_labels=node_labels)

T = G.get_spanning_tree()
N = Net(P, T).unfold().in_own_span()

2022-07-03 14:47:17.074 | DEBUG    | tope.tope:from_vertices:39 - Computing from set of 24 vertices.
2022-07-03 14:47:17.091 | DEBUG    | tope.tope:from_vertices:46 - Found 24 facets.
2022-07-03 14:47:17.145 | DEBUG    | tope.tope:from_vertices:87 - Finished processing faces:
 [[{0}, {1}, {2}, {3}, {4}, {5}, {6}, {7}, {8}, {9}, {10}, {11}, {12}, {13}, {14}, {15}, {16}, {17}, {18}, {19}, {20}, {21}, {22}, {23}], [{0, 21}, {17, 10}, {8, 21}, {8, 17}, {21, 6}, {17, 6}, {17, 21}, {17, 5}, {17, 3}, {17, 13}, {17, 23}, {13, 23}, {3, 23}, {1, 11}, {1, 23}, {4, 21}, {21, 15}, {19, 21}, {21, 23}, {19, 23}, {15, 23}, {11, 23}, {5, 13}, {1, 9}, {1, 13}, {19, 6}, {13, 6}, {19, 13}, {13, 14}, {9, 13}, {9, 11}, {19, 4}, {11, 19}, {2, 19}, {9, 19}, {3, 5}, {1, 7}, {1, 3}, {1, 20}, {1, 5}, {8, 15}, {8, 3}, {3, 15}, {18, 3}, {3, 7}, {11, 7}, {4, 15}, {11, 15}, {16, 11}, {11, 4}, {12, 15}, {7, 15}, {20, 7}, {18, 7}, {16, 7}, {12, 7}, {9, 20}, {9, 14}, {16, 9}, {9, 2}, {16, 20}, {10, 5}, {5, 14}, {18, 5}

In [31]:
# need to locate external faces
# any faces not marked by an edge is external
verts = np.concatenate(list(N.facets.values()))

In [32]:
faces = []
for k in N.facets:
    Fk = P.get_facet(k)
    faces.append(np.stack([N.facets[k][sorted(I)] for I in Fk.faces[-1]])) # all have length 3
faces = np.concatenate(faces)

In [33]:
thing = stl.mesh.Mesh(np.zeros(192, dtype=stl.mesh.Mesh.dtype))
thing.vectors[:,:,:] = faces
thing.save('thing.stl')